In [ ]:
import yaml
import pickle
import os.path as osp
import subprocess
from urllib.parse import urlparse

In [ ]:
def retrieve_url(url, dest_path, force_download=False):
    if not osp.exists(dest_path) or force_download:
        target_extension = osp.splitext(dest_path)[1]
        url_extension = osp.splitext(urlparse(url).path)[1]
        if target_extension != url_extension:
            print("Warning: file extension from url does not match destination file extension")
        subprocess.call(f"wget -O {dest_path}  {url}", shell=True)
        assert osp.exists(dest_path)
        print(f"Successfully downloaded {url} to {dest_path}")
    else:
        print(f"Target data already exists at {dest_path}")

In [ ]:
def read_pkl(file_path):
    with open(file_path, 'rb') as file:
        print(f"loading file {file_path}")
        d = pickle.load(file)
    return d

In [ ]:
def read_yml(yaml_path, subkey=None):
    with open(yaml_path, 'r') as file:
        d = yaml.safe_load(file)
        if subkey is not None:
            d = d[subkey]
    return d

In [ ]:
def foo():
    d = read_pkl()
    dat = format_fmda_data()
    dat = filter_data(dat)
    return dat

In [ ]:
def format_fmda_data(d)
    # Inputs:
    #    d: (dict) fmda dictionary, output of process in wrfxpy

    return d

In [ ]:
read_yml("params_data.yaml")

In [ ]:
file = 'data/test_CA_202401.pkl'

In [ ]:
d = read_pkl(file)
d